# Experiments
TODO: 24-27 June 2019:
* Create pipeline
    * with initial unigrams baseline
    * accuracy measure (e.g. precision-recall with AUROC)
## Setup environment:
* import libraries
* load csv data

In [1]:
import random
import pandas as pd
from nltk.corpus import treebank
description_df = pd.read_csv('./data/description.csv')
installation_df = pd.read_csv('./data/installation.csv')
invocation_df = pd.read_csv('./data/invocation.csv')
citation_df = pd.read_csv('./data/citation.csv')

## Data Preview
Make sure that csv data has been successfully imported.

In [2]:
description_df.head()

,URL,excerpt
0,https://github.com/GoogleChrome/puppeteer,Puppeteer is a Node library which provides a h...
1,https://github.com/JimmySuen/integral-human-pose,The major contributors of this repository incl...
2,https://github.com/JimmySuen/integral-human-pose,Integral Regression is initially described in ...
3,https://github.com/JimmySuen/integral-human-pose,We build a 3D pose estimation system based mai...
4,https://github.com/JimmySuen/integral-human-pose,The Integral Regression is also known as soft-...


In [3]:
installation_df.head()

,URL,excerpt
0,https://github.com/GoogleChrome/puppeteer,Installation
1,https://github.com/GoogleChrome/puppeteer,"To use Puppeteer in your project, run:"
2,https://github.com/GoogleChrome/puppeteer,npm i puppeteer
3,https://github.com/GoogleChrome/puppeteer,"# or ""yarn add puppeteer"""
4,https://github.com/GoogleChrome/puppeteer,puppeteer-core


In [4]:
invocation_df.head()

,URL,excerpt
0,https://github.com/JimmySuen/integral-human-pose,Usage
1,https://github.com/JimmySuen/integral-human-pose,We have placed some example config files in ex...
2,https://github.com/JimmySuen/integral-human-pose,Train
3,https://github.com/JimmySuen/integral-human-pose,"For Integral Human Pose Regression, cd to pyto..."
4,https://github.com/JimmySuen/integral-human-pose,Integral Regression


In [5]:
citation_df.head()

,URL,excerpt
0,https://github.com/JimmySuen/integral-human-pose,If you find Integral Regression useful in your...
1,https://github.com/JimmySuen/integral-human-pose,"@article{sun2017integral,"
2,https://github.com/JimmySuen/integral-human-pose,"title={Integral human pose regression},"
3,https://github.com/JimmySuen/integral-human-pose,"author={Sun, Xiao and Xiao, Bin and Liang, Shu..."
4,https://github.com/JimmySuen/integral-human-pose,"journal={arXiv preprint arXiv:1711.08229},"


Each data set currently contains positive samples of its respective trait. However, negative samples are necessary to distinguish the positive against some sort of control. Per category, negative samples include those from the other categories and also text samples completely unrelated to repository information. For example, in the description classifier, positive samples would be those that were labelled as a description, and negative samples would include those labelled as a installation, invocation, or citation in addition to nonpertinent text such as the Treebank corpus.

As there are many more negative samples than there are positive samples, randomly selected negative samples will be used. The aim is for about 60% positive and 40% negative. Of the 40% negative, 10% for each outside category and 10% for random, e.g. Treebank, text. 

## Description Classifier

In [6]:
neg_quant = int(len(description_df) / 6)
treebank_background = pd.DataFrame(list(map(lambda sent: ' '.join(sent), random.sample(list(treebank.sents()), neg_quant))), columns=["excerpt"]).assign(description=False)
train_description = pd.concat([description_df.assign(description=True), installation_df.sample(neg_quant).assign(description=False), invocation_df.sample(neg_quant).assign(description=False), citation_df.sample(neg_quant).assign(description=False),treebank_background], sort=False)
train_description.drop('URL', 1, inplace=True)
train_description.reset_index(drop=True, inplace=True)
train_description

,excerpt,description
0,Puppeteer is a Node library which provides a h...,True
1,The major contributors of this repository incl...,True
2,Integral Regression is initially described in ...,True
3,We build a 3D pose estimation system based mai...,True
4,The Integral Regression is also known as soft-...,True
5,This is an official implementation for Integra...,True
6,The original implementation is based on our in...,True
7,LibGEOS is a LGPL-licensed package for manipul...,True
8,"Among other things, it allows you to parse Wel...",True
9,This repository contains the experiments in th...,True


Note: since Treebank corpus sentences have already been split into words, they will be used directly later.